In [ ]:
%%capture
import os

# Install dependencies for TEXT-ONLY fine-tuning
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2
!pip install pandas numpy tqdm scikit-learn

print("✅ All dependencies installed for TEXT-ONLY training!")

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

print("✅ Libraries imported!")
print("📝 MODE: TEXT-ONLY price prediction (no images)")
print("⚡ Expected training time: 4-6 hours for 75K samples")

In [ ]:
# ===============================
# ⚙️ CONFIGURATION
# ===============================

# Paths
DATASET_FOLDER = '/kaggle/input/amazon-ml-challenge-2025/student_resource/dataset'

# Model (TEXT-ONLY)
MODEL_NAME = "unsloth/Llama-3.2-3B"  # 7B parameter model

# Training (OPTIMIZED FOR A100 80GB)
PER_DEVICE_BATCH_SIZE = 8  # Large batch for A100
GRADIENT_ACCUMULATION_STEPS = 2  # Effective batch = 16
NUM_EPOCHS = 2  # 2 epochs for 75K samples
LEARNING_RATE = 2e-4
WARMUP_RATIO = 0.03
MAX_SEQ_LENGTH = 2048  # Maximum sequence length

# LoRA
LORA_R = 16
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

# Data
VALIDATION_SPLIT = 0.2  # 80/20 train/val
SAMPLE_SIZE = None  # None = all data, or set number for testing (e.g., 1000)

# Output
OUTPUT_DIR = "qwen_text_price_model"

# Inference
TEMPERATURE = 0.1  # Low temp for consistent numeric output
MAX_NEW_TOKENS = 20  # Just need the price number

print(f"✅ Configuration:")
print(f"   Model: {MODEL_NAME}")
print(f"   Batch size: {PER_DEVICE_BATCH_SIZE} (effective: {PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS})")
print(f"   Epochs: {NUM_EPOCHS}")
print(f"   Max length: {MAX_SEQ_LENGTH}")
print(f"   TEXT-ONLY: No images required!")

# Load data
print(f"\n📂 Loading data...")
train_full = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))

print(f"✓ Train: {len(train_full):,} rows")
print(f"✓ Test: {len(test):,} rows")

# Sample for testing (optional)
if SAMPLE_SIZE is not None:
    train_full = train_full.sample(n=min(SAMPLE_SIZE, len(train_full)), random_state=42).reset_index(drop=True)
    print(f"⚠️  Using sample: {len(train_full):,} rows for testing")

# Train/Val split
train, val = train_test_split(
    train_full, 
    test_size=VALIDATION_SPLIT, 
    random_state=42
)

train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

print(f"\n📊 Split:")
print(f"   Train: {len(train):,} rows")
print(f"   Val: {len(val):,} rows")
print(f"\n💲 Price statistics:")
print(f"   Min: ${train_full['price'].min():.2f}")
print(f"   Max: ${train_full['price'].max():.2f}")
print(f"   Mean: ${train_full['price'].mean():.2f}")
print(f"   Median: ${train_full['price'].median():.2f}")

## 🎨 Optimized Prompt Engineering for Price Prediction

**TEXT-ONLY approach**: Using only catalog content to predict prices.
No images needed - faster training and inference!

In [ ]:
# ===============================
# 🎯 OPTIMIZED PROMPT FOR TEXT-ONLY PRICE PREDICTION
# ===============================

INSTRUCTION = """You are an expert price prediction system for e-commerce products. Analyze the product catalog description and predict the price in USD.

CRITICAL RULES:
1. Output ONLY a numeric price value (e.g., 12.99)
2. NO dollar signs ($), NO currency symbols, NO text explanations
3. DO NOT confuse quantity/weight with price (e.g., "16 oz" ≠ $16.00)
4. Consider these factors:
   - Brand reputation (premium brands = higher prices)
   - Product type and category
   - Pack quantity (multi-packs = higher total price)
   - Quality indicators (organic, premium, luxury)
   - Package size and unit quantity

5. Typical price ranges:
   - Food items: $2-50
   - Beverages: $1-30
   - Beauty/Health: $5-100
   - Electronics: $10-500
   - Home goods: $5-200

OUTPUT FORMAT: Just the number
CORRECT: 14.99
WRONG: $14.99, "The price is 14.99", "14.99 dollars"
"""

def convert_to_conversation(sample):
    """
    Convert sample to TEXT-ONLY conversation format.
    NO IMAGES - just text catalog content.
    """
    # Build conversation with text only
    conversation = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text", 
                    "text": f"{INSTRUCTION}\n\nPRODUCT DETAILS:\n{sample['catalog_content']}\n\nPredicted Price:"
                }
            ]
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text", 
                    "text": f"{sample['price']:.2f}"  # Format: "12.99"
                }
            ]
        }
    ]
    
    return {"messages": conversation}

print("✅ TEXT-ONLY prompt template defined!")
print("\n📝 Key improvements:")
print("   - Clear anti-hallucination instructions")
print("   - Prevents quantity/price confusion")
print("   - Enforces numeric-only output")
print("   - Provides price range context")
print("   - No images required!")

In [ ]:
# Show sample catalog content
print("📋 Sample catalog content:")
print("="*70)
print(train.loc[0, "catalog_content"][:300])
print("...")
print("="*70)
print(f"\n💲 Actual price: ${train.loc[0, 'price']:.2f}")

'Item Name: La Victoria Green Taco Sauce Mild, 12 Ounce (Pack of 6)\nValue: 72.0\nUnit: Fl Oz\n'

In [ ]:
# ===============================
# 🔄 Convert datasets to conversation format with proper tokenization
# ===============================

print("🔄 Converting training data to conversation format...")
print(f"   Processing {len(train):,} samples\n")

def format_prompt_for_training(sample):
    """Format sample for training with proper structure."""
    # Create the full prompt
    prompt = f"{INSTRUCTION}\n\nPRODUCT DETAILS:\n{sample['catalog_content']}\n\nPredicted Price:"
    response = f"{sample['price']:.2f}"
    
    # Manually format text (no chat template needed)
    text = f"{prompt} {response}"
    
    return {"text": text}

train_dataset = []
for idx, row in enumerate(train.itertuples(index=False)):
    sample = {
        "catalog_content": row.catalog_content,
        "price": row.price
    }
    train_dataset.append(format_prompt_for_training(sample))
    
    # Progress indicator
    if (idx + 1) % 10000 == 0:
        print(f"   Processed: {idx + 1:,}/{len(train):,}")

print(f"\n✅ Train dataset ready: {len(train_dataset):,} samples")

# Convert validation data
print(f"\n🔄 Converting validation data...")
val_dataset = []
for row in val.itertuples(index=False):
    sample = {
        "catalog_content": row.catalog_content,
        "price": row.price
    }
    val_dataset.append(format_prompt_for_training(sample))

print(f"✅ Validation dataset ready: {len(val_dataset):,} samples")

In [ ]:
# Show example conversation
print("📋 Example training text:")
print("="*70)
print(train_dataset[0]['text'][:500])
print("...")
print("\n" + "="*70)

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': "\nYou are a helpful assistant. \nThe user provides a product catalog and an image. Your task is to predict the price in USD. \n\nImportant:\n- Do NOT use numbers from weights, volumes, or quantities (like '12.7 Ounce', '11.25 oz') as the price. \n- Consider the type of product, brand, and image cues to estimate the correct price. \n- Output ONLY the price as a number in USD. \n- Example: 4.89\n\n\nItem Name: La Victoria Green Taco Sauce Mild, 12 Ounce (Pack of 6)\nValue: 72.0\nUnit: Fl Oz\n"},
    {'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=1000x1000>}]},
  {'role': 'assistant',
   'content': [{'type': 'text', 'text': '4.890000000000001'}]}]}

### Unsloth

In [ ]:
from unsloth import FastLanguageModel  # Use FastLanguageModel for TEXT-ONLY
import torch

print(f"🤖 Loading TEXT-ONLY model: {MODEL_NAME}")
print("   This will take 2-3 minutes...\n")

# Load model with Unsloth (TEXT-ONLY, no vision)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect
    load_in_4bit=True,  # 4-bit quantization for memory efficiency
)

print("✅ Model loaded successfully!")
print(f"   Model: Qwen2.5-7B-Instruct (TEXT-ONLY)")
print(f"   Parameters: ~7B")
print(f"   Max sequence length: {MAX_SEQ_LENGTH}")
print(f"   Quantization: 4-bit")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-10-12 09:57:36.548679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760263056.575221    2043 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760263056.583718    2043 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `ty

🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.1: Fast Qwen2_5_Vl patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

**[NEW]** We also support finetuning ONLY the vision part of the model, or ONLY the language part. Or you can select both! You can also select to finetune the attention or the MLP layers!

In [ ]:
print("🎯 Adding LoRA adapters for efficient fine-tuning...\n")

# Add LoRA adapters - only train ~1% of parameters
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized checkpointing
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

print("✅ LoRA adapters added!")
print(f"   Rank (r): {LORA_R}")
print(f"   Alpha: {LORA_ALPHA}")
print(f"   Dropout: {LORA_DROPOUT}")
print(f"   Trainable parameters: ~1% of total (very efficient!)")

In [ ]:
from transformers import TextStreamer

print("🧪 Testing model BEFORE fine-tuning...\n")

FastLanguageModel.for_inference(model)

# Test sample
test_prompt = f"{INSTRUCTION}\n\nPRODUCT DETAILS:\n{train.loc[0, 'catalog_content']}\n\nPredicted Price:"

# Tokenize
inputs = tokenizer(
    test_prompt,
    return_tensors="pt",
    truncation=True,
    max_length=MAX_SEQ_LENGTH
).to("cuda")

# Generate
print("🔮 Pre-training prediction:")
text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
    do_sample=False,  # Greedy for testing
)

actual_price = train.loc[0, 'price']
print(f"\n✓ Actual price: ${actual_price:.2f}")
print("\n💡 After fine-tuning, predictions should match actual prices closely!")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

24.0<|im_end|>


In [ ]:
from trl import SFTTrainer, SFTConfig

print("🏋️ Setting up trainer for TEXT-ONLY fine-tuning...")
print("="*70)

# Calculate training metrics
total_samples = len(train_dataset)
effective_batch_size = PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS
steps_per_epoch = total_samples // effective_batch_size
total_steps = steps_per_epoch * NUM_EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)

print(f"📊 Training Configuration:")
print(f"   Train samples: {total_samples:,}")
print(f"   Val samples: {len(val_dataset):,}")
print(f"   Batch size: {PER_DEVICE_BATCH_SIZE}")
print(f"   Gradient accumulation: {GRADIENT_ACCUMULATION_STEPS}")
print(f"   Effective batch: {effective_batch_size}")
print(f"   Steps per epoch: {steps_per_epoch:,}")
print(f"   Total epochs: {NUM_EPOCHS}")
print(f"   Total steps: {total_steps:,}")
print(f"   Warmup steps: {warmup_steps:,}")
print(f"\n⏱️  Estimated time: {total_steps * 1.5 / 3600:.1f} hours on A100 80GB")
print("="*70 + "\n")

# Enable training mode
FastLanguageModel.for_training(model)

# Create trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # For validation
    dataset_text_field="text",  # Use 'text' field from our formatted data
    max_seq_length=MAX_SEQ_LENGTH,
    args=SFTConfig(
        # Batch & optimization
        per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
        per_device_eval_batch_size=PER_DEVICE_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        
        # Training length
        num_train_epochs=NUM_EPOCHS,
        max_steps=-1,  # Use epochs
        
        # Learning rate
        learning_rate=LEARNING_RATE,
        warmup_steps=warmup_steps,
        lr_scheduler_type="cosine",
        
        # Optimizer
        optim="adamw_8bit",  # 8-bit Adam for memory
        weight_decay=0.01,
        
        # Logging & evaluation
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=500,
        save_strategy="steps",
        save_steps=1000,
        save_total_limit=2,
        
        # Output
        output_dir=OUTPUT_DIR,
        report_to="none",
        
        # Performance
        fp16=True,  # Mixed precision
        seed=42,
    ),
)

print("✅ Trainer configured successfully!")
print("   Ready to start training...\n")

Unsloth: Model does not have a default image size - using 512


In [19]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.611 GB of memory reserved.


In [ ]:
# START TRAINING!
print("\n🚀 Starting training...")
print("   Monitor GPU with: watch -n 1 nvidia-smi\n")

trainer_stats = trainer.train()

print("\n✅ Training complete!")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10 | Num Epochs = 15 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,084,928 of 3,795,707,904 (1.08% trained)


Step,Training Loss
1,1.249400
2,0.560800
3,1.068600
4,1.294100
5,1.127700
6,0.685000
7,1.066200
8,0.401100
9,0.987600
10,0.823900


In [24]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

217.4169 seconds used for training.
3.62 minutes used for training.
Peak reserved memory = 4.344 GB.
Peak reserved memory for training = 0.733 GB.
Peak reserved memory % of max memory = 29.469 %.
Peak reserved memory for training % of max memory = 4.973 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [ ]:
import re
from tqdm.auto import tqdm

def parse_price_output(text):
    """Robust price parsing from model output."""
    try:
        # Remove common prefixes/suffixes
        text = text.replace('$', '').replace('USD', '').replace('Price:', '').strip()
        
        # Find first number
        match = re.search(r'\d+\.?\d*', text)
        if match:
            price = float(match.group())
            # Sanity check (0.01 to 10000)
            if 0.01 <= price <= 10000:
                return price
        
        return None
    except:
        return None

def calculate_smape(actual, predicted):
    """Calculate SMAPE."""
    return np.mean(np.abs(predicted - actual) / ((np.abs(actual) + np.abs(predicted)) / 2)) * 100

print("🎯 Evaluating on validation set...")
print(f"   {len(val_dataset):,} samples")
print("   This will take 10-20 minutes\n")

FastLanguageModel.for_inference(model)

predictions = []
actuals = []
failed_parses = 0

for idx in tqdm(range(min(len(val_dataset), 1000)), desc="Validating"):  # Test on 1000 samples first
    # Get catalog content
    catalog_content = val.iloc[idx]['catalog_content']
    actual_price = val.iloc[idx]['price']
    
    # Build prompt
    test_prompt = f"{INSTRUCTION}\n\nPRODUCT DETAILS:\n{catalog_content}\n\nPredicted Price:"
    
    # Tokenize
    inputs = tokenizer(
        test_prompt,
        return_tensors="pt",
        truncation=True,
        max_length=MAX_SEQ_LENGTH
    ).to("cuda")
    
    # Generate
    output = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW_TOKENS,
        temperature=TEMPERATURE,
        do_sample=False,
    )
    
    predicted_text = tokenizer.decode(output[0], skip_special_tokens=True)
    predicted_price = parse_price_output(predicted_text)
    
    if predicted_price is None:
        failed_parses += 1
        predicted_price = actual_price  # Fallback
    
    predictions.append(predicted_price)
    actuals.append(actual_price)

# Calculate SMAPE
predictions = np.array(predictions)
actuals = np.array(actuals)
val_smape = calculate_smape(actuals, predictions)

print("\n" + "="*70)
print("📊 VALIDATION RESULTS")
print("="*70)
print(f"\n✅ Validation SMAPE: {val_smape:.2f}%")
print(f"   Failed parses: {failed_parses}/{len(predictions)} ({100*failed_parses/len(predictions):.1f}%)")
print(f"\n🎯 Target: < 45% test SMAPE")
print(f"   Your validation: {val_smape:.2f}%")

if val_smape < 45:
    print("\n🎉 EXCELLENT! Below target!")
elif val_smape < 50:
    print("\n✅ Good! Close to target")
else:
    print("\n⚠️  Needs improvement - consider more training")

3.20<|im_end|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
print("💾 Saving fine-tuned model...\n")

# Save LoRA adapters
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Model saved to {OUTPUT_DIR}")
print("   Contains LoRA adapters only (small size)")

## 🎯 Generate Test Predictions

Now generate predictions for the test set

In [ ]:
print("🎯 Generating test predictions...")
print(f"   {len(test):,} samples")
print("   This will take 1-2 hours\n")

FastLanguageModel.for_inference(model)

test_predictions = []
failed_parses_test = 0

for idx in tqdm(range(len(test)), desc="Test inference"):
    catalog_content = test.iloc[idx]['catalog_content']
    
    # Build prompt
    test_prompt = f"{INSTRUCTION}\n\nPRODUCT DETAILS:\n{catalog_content}\n\nPredicted Price:"
    
    # Tokenize
    inputs = tokenizer(
        test_prompt,
        return_tensors="pt",
        truncation=True,
        max_length=MAX_SEQ_LENGTH
    ).to("cuda")
    
    # Generate
    output = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW_TOKENS,
        temperature=TEMPERATURE,
        do_sample=False,
    )
    
    predicted_text = tokenizer.decode(output[0], skip_special_tokens=True)
    predicted_price = parse_price_output(predicted_text)
    
    if predicted_price is None:
        failed_parses_test += 1
        predicted_price = 10.0  # Default fallback
    
    test_predictions.append(predicted_price)

print(f"\n✅ Test predictions generated!")
print(f"   Failed parses: {failed_parses_test}/{len(test)} ({100*failed_parses_test/len(test):.1f}%)")

# Create submission
submission = pd.DataFrame({
    'sample_id': test['sample_id'],
    'price': test_predictions
})

# Save
submission_file = 'submission_qwen_text_only.csv'
submission.to_csv(submission_file, index=False)

print(f"\n💾 Submission saved: {submission_file}")
print(f"   Shape: {submission.shape}")
print(f"\n📊 Price statistics:")
print(f"   Min: ${submission['price'].min():.2f}")
print(f"   Max: ${submission['price'].max():.2f}")
print(f"   Mean: ${submission['price'].mean():.2f}")
print(f"   Median: ${submission['price'].median():.2f}")

submission.head(10)

## 📊 Final Summary

In [ ]:
print("\n" + "="*70)
print("🎉 QWEN2.5-7B TEXT-ONLY FINE-TUNING COMPLETE!")
print("="*70)

print(f"\n📊 Results:")
print(f"   Validation SMAPE: {val_smape:.2f}%")
print(f"   Test predictions: {len(test_predictions):,}")
print(f"   Submission file: {submission_file}")

print(f"\n⏱️  Time spent:")
print(f"   Training: {trainer_stats.metrics['train_runtime']/3600:.1f} hours")
print(f"   Validation: ~0.3 hours")
print(f"   Test inference: ~1-2 hours")

print(f"\n🎯 Next steps:")
print(f"   1. Upload {submission_file} to competition")
print(f"   2. Check test SMAPE on leaderboard")
print(f"   3. Compare to validation SMAPE ({val_smape:.2f}%)")

print(f"\n💡 Expected outcome:")
if val_smape < 45:
    print(f"   ✅ You should be competitive! (< 45% target)")
    print(f"   Test SMAPE likely: {val_smape:.1f}% - {val_smape+3:.1f}%")
elif val_smape < 50:
    print(f"   ⚠️  Close but may need iteration")
    print(f"   Test SMAPE likely: {val_smape:.1f}% - {val_smape+5:.1f}%")
else:
    print(f"   ❌ May need different approach")
    print(f"   Consider brand-focused solution instead")

print("\n" + "="*70)
print("Good luck! 🚀")
print("="*70)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
